In [1]:
import helper_func as hf
import cv2
import numpy as np
import pandas as pd
import os
from pprint import pprint 
from matplotlib import pyplot as plt

overview = os.path.join('.', 'data', 'overview')
focus = os.path.join('.', 'data', 'focus')
cnn = os.path.join('.', 'data', 'CNN_data')
cnn_cut = os.path.join('.', 'data', 'cnn_data_cut')

images = hf.load_data(overview, grayscale=False)
# images = hf.load_data(focus, grayscale=False)
images = hf.load_data(cnn_data, grayscale=False)
img = images[0]['img']

Loaded (8) images from .\data\overview


In [2]:
import math
from PIL import Image, ImageStat

# Convert image to greyscale, return average pixel brightness.
def brightness_1( im_file ):
   im = Image.open(im_file).convert('L')
   stat = ImageStat.Stat(im)
   return stat.mean[0]

# Convert image to greyscale, return RMS pixel brightness.
def brightness_2( im_file ):
   im = Image.open(im_file).convert('L')
   stat = ImageStat.Stat(im)
   return stat.rms[0]

# Average pixels -> perceived brightness.
def brightness_3( im_file ):
   im = Image.open(im_file)
   stat = ImageStat.Stat(im)
   r,g,b = stat.mean
   return math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))

# RMS of pixels -> perceived brightness.
def brightness_4( im_file ):
   im = Image.open(im_file)
   stat = ImageStat.Stat(im)
   r,g,b = stat.rms
   return math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))

# Calculate PB of pixels, then return average.
def brightness_5( im_file ):
   im = Image.open(im_file)
   stat = ImageStat.Stat(im)
   gs = (math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2)) 
         for r,g,b in im.getdata())
   return sum(gs)/stat.count[0]

In [3]:
import numpy as np
import cv2
from collections import namedtuple

#brange brightness range
#bval brightness value
BLevel = namedtuple("BLevel", ['brange', 'bval'])

#all possible levels
_blevels = [
    BLevel(brange=range(0, 24), bval=0),
    BLevel(brange=range(23, 47), bval=1),
    BLevel(brange=range(46, 70), bval=2),
    BLevel(brange=range(69, 93), bval=3),
    BLevel(brange=range(92, 116), bval=4),
    BLevel(brange=range(115, 140), bval=5),
    BLevel(brange=range(139, 163), bval=6),
    BLevel(brange=range(162, 186), bval=7),
    BLevel(brange=range(185, 209), bval=8),
    BLevel(brange=range(208, 232), bval=9),
    BLevel(brange=range(231, 256), bval=10),
]


def detect_level(h_val):
    h_val = int(h_val)
    for blevel in _blevels:
        if h_val in blevel.brange:
            return blevel.bval
    raise ValueError("Brightness Level Out of Range")


def get_img_avg_brightness(image_path):
    img = cv2.imread(image_path)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    _, _, v = cv2.split(hsv)

    return int(np.average(v.flatten()))

In [4]:
## OpenCV
# Intesity of individual pixel are img[x,y] greyscale every channel have the same value
# doc -> https://docs.opencv.org/3.4/d5/d98/tutorial_mat_operations.html

def opencv_intensity(image_path):
    img = cv2.imread(image_path)
    height,width,channels = img.shape
    intensity = 0
    for x in range(height):
        for y in range(width):
            intensity += img[x, y][0]
    return intensity/(height*width)

In [5]:
images_cut = os.listdir(cnn_data)
# images = hf.load_data(cnn_data, grayscale=False)
brightness = {
    'True': {
        'name': [],
        'APB': [],
        'RMS': [],
        'AP_PB': [],
        'RMS_PB': [],
        'PB_AVG': [],
        'intesity': [],
        'brightnessLVL': []
    },
    'False': {
        'name': [],
        'APB': [],
        'RMS': [],
        'AP_PB': [],
        'RMS_PB': [],
        'PB_AVG': [],
        'intesity': [],
        'brightnessLVL': []
    }
}

for image in images_cut:
    print('\rImage to process {}'.format(image), end='')
    image_path = os.path.join(cnn_cut, image)
    brightness[image.split('_')[1]]['name'].append(image)
    brightness[image.split('_')[1]]['APB'].append(round(brightness_1(image_path), 4))
    brightness[image.split('_')[1]]['RMS'].append(round(brightness_2(image_path), 4))
    brightness[image.split('_')[1]]['AP_PB'].append(round(brightness_3(image_path), 4))
    brightness[image.split('_')[1]]['RMS_PB'].append(round(brightness_4(image_path), 4))
    brightness[image.split('_')[1]]['PB_AVG'].append(round(brightness_5(image_path), 4))
    brightness[image.split('_')[1]]['intesity'].append(opencv_intensity(image_path))
    brightness[image.split('_')[1]]['brightnessLVL'].append(detect_level(get_img_avg_brightness(image_path)))

Image to process 13_True_2x2.png

In [6]:
import pandas as pd

positive = pd.DataFrame(brightness['True'])
negative = pd.DataFrame(brightness['False'])

In [7]:
positive.describe()

,APB,RMS,AP_PB,RMS_PB,PB_AVG,intesity,brightnessLVL
count,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000
mean,109.814711,113.572003,109.814711,113.572003,109.814711,109.814712,4.236842
std,18.682127,17.156364,18.682127,17.156364,18.682127,18.682127,0.819829
min,69.407800,71.632000,69.407800,71.632000,69.407800,69.407782,2.000000
25%,98.449250,103.868350,98.449250,103.868350,98.449250,98.449262,4.000000
50%,114.923050,115.797650,114.923050,115.797650,114.923050,114.923022,4.000000
75%,122.846075,125.324050,122.846075,125.324050,122.846075,122.846075,5.000000
max,139.290900,140.974500,139.290900,140.974500,139.290900,139.290857,5.000000


In [8]:
negative.describe()

,APB,RMS,AP_PB,RMS_PB,PB_AVG,intesity,brightnessLVL
count,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000
mean,98.941190,107.254386,98.941190,107.254386,98.941190,98.941192,3.711864
std,18.637087,19.119303,18.637087,19.119303,18.637087,18.637081,0.851993
min,61.674300,70.487800,61.674300,70.487800,61.674300,61.674325,2.000000
25%,84.300200,91.306900,84.300200,91.306900,84.300200,84.300208,3.000000
50%,97.844900,107.109500,97.844900,107.109500,97.844900,97.844875,4.000000
75%,113.536200,122.038750,113.536200,122.038750,113.536200,113.536201,4.000000
max,138.628000,140.980700,138.628000,140.980700,138.628000,138.627958,5.000000


In [11]:
negative.head(10)

,name,APB,RMS,AP_PB,RMS_PB,PB_AVG,intesity,brightnessLVL
0,02_False_0x0.png,104.1247,122.0600,104.1247,122.0600,104.1247,104.124660,4
1,02_False_0x1.png,96.1122,111.6924,96.1122,111.6924,96.1122,96.112249,4
2,02_False_0x2.png,99.1276,115.7044,99.1276,115.7044,99.1276,99.127628,4
3,02_False_0x3.png,102.3098,119.4820,102.3098,119.4820,102.3098,102.309789,4
4,02_False_1x0.png,108.3798,124.1847,108.3798,124.1847,108.3798,108.379839,4
5,02_False_1x1.png,91.0809,100.2314,91.0809,100.2314,91.0809,91.080892,3
6,02_False_1x2.png,93.5875,102.2597,93.5875,102.2597,93.5875,93.587543,4
7,02_False_1x3.png,102.0164,112.7707,102.0164,112.7707,102.0164,102.016440,4
8,02_False_2x0.png,122.5596,137.5068,122.5596,137.5068,122.5596,122.559554,5
9,02_False_2x1.png,111.7200,122.0175,111.7200,122.0175,111.7200,111.720031,4
